#  ЗАДАНИЕ
1. Посчитать показатели Retention Rate седьмого дня и ARPPU за два периода: до и после внедрения фичи.
2. Сформировать выводы по анализу.

In [2]:
import json 
import requests
from pprint import pprint
import pandas as pd
import matplotlib.pyplot as plt
import plotly
import plotly.express as px
%matplotlib inline
import numpy as np
import psycopg2
import psycopg2.extras

In [3]:
# Функция getRegisteredUsersCount выполняет SQL запрос подсчетом уникальных пользователей по дням
def getRegisteredUsersCount():
    query = '''SELECT r.registration_date, count(distinct(r.user_id)) as registered_users_count
    FROM case9.registration r
    WHERE r.registration_date >= '2019-05-01' and r.registration_date < '2019-07-01'
    GROUP BY 1
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data

In [4]:
# Записываем данные в ДатаФрейм
registered_users_count = pd.DataFrame(getRegisteredUsersCount())

registered_users_count.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 2 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   registration_date       61 non-null     object
 1   registered_users_count  61 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.1+ KB


In [5]:
# Меняем тип данных в колонке registration_date с object на datetime
registered_users_count['registration_date'] = pd.to_datetime(registered_users_count['registration_date'])

In [7]:
# Посмотрим на данные
registered_users_count.head()

,registration_date,registered_users_count
0,2019-05-01,6516
1,2019-05-02,5309
2,2019-05-03,6266
3,2019-05-04,8639
4,2019-05-05,9787


In [8]:
# Функция getActiveUsersCountWithCohorts выполняет SQL запрос в базу данных для формирования датафрема с активными пользователями
def getActiveUsersCountWithCohorts():
    query = '''SELECT ac.activity_date, r.registration_date, count(distinct(ac.user_id)) as active_users_count
    FROM case9.user_activity ac
    LEFT JOIN case9.registration r ON r.user_id = ac.user_id
    WHERE r.registration_date >= '2019-05-01' and r.registration_date < '2019-07-01'
    GROUP BY 1,2
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data

# Запишем  данные по пользовотелям в датафрейм active_users_count_with_cohorts
active_users_count_with_cohorts = pd.DataFrame(getActiveUsersCountWithCohorts())

# Посмотрим на инофрмацию о датафрейме
active_users_count_with_cohorts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1891 entries, 0 to 1890
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   activity_date       1891 non-null   object
 1   registration_date   1891 non-null   object
 2   active_users_count  1891 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 44.4+ KB


In [9]:
# Ихменим тип данных в первых двух колонках
for col in ['activity_date','registration_date']:
    active_users_count_with_cohorts[col] = pd.to_datetime(active_users_count_with_cohorts[col])

In [10]:
# Объединим таблицы active_users_count_with_cohorts и registered_users_count по стобцу registration_date
retention_table = active_users_count_with_cohorts.merge(registered_users_count,on=['registration_date'],how='left')

#Посмотрим на данные
retention_table.head()

,activity_date,registration_date,active_users_count,registered_users_count
0,2019-05-01,2019-05-01,2590,6516
1,2019-05-02,2019-05-01,2146,6516
2,2019-05-02,2019-05-02,2049,5309
3,2019-05-03,2019-05-01,1637,6516
4,2019-05-03,2019-05-02,1817,5309


In [11]:
# Добавим расчетные данные в объединенный датафремй - метрику lifetime

retention_table['lifetime'] = retention_table['activity_date'] - retention_table['registration_date']
retention_table['lifetime'] = retention_table['lifetime']/np.timedelta64(1,'D')
retention_table['lifetime'] = retention_table['lifetime'].astype(int) # Приведем тип к целому числу
retention_table.head()

,activity_date,registration_date,active_users_count,registered_users_count,lifetime
0,2019-05-01,2019-05-01,2590,6516,0
1,2019-05-02,2019-05-01,2146,6516,1
2,2019-05-02,2019-05-02,2049,5309,0
3,2019-05-03,2019-05-01,1637,6516,2
4,2019-05-03,2019-05-02,1817,5309,1


Добавляем к этой таблице данные по доходам

In [16]:
# Созданим функцию и запишем ее даные в датафорем с доходами revenue

def getRevenue():
    query = '''SELECT re.date, r.registration_date, count(distinct(re.user_id)) as users_count_with_revenue, sum(re.revenue) as revenue
    FROM case9.user_revenue re
    LEFT JOIN case9.registration r ON r.user_id = re.user_id
    WHERE r.registration_date >= '2019-05-01' and r.registration_date < '2019-07-01'
    GROUP BY 1,2
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data

revenue = pd.DataFrame(getRevenue())

revenue.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1745 entries, 0 to 1744
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   date                      1745 non-null   object 
 1   registration_date         1745 non-null   object 
 2   users_count_with_revenue  1745 non-null   int64  
 3   revenue                   1745 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 54.7+ KB


In [17]:
# Преобразуем типы данных в таблице revenue
for col in ['date','registration_date']:
    revenue[col] = pd.to_datetime(revenue[col])
    
# Посмотрим на таблицу
revenue.head()

,date,registration_date,users_count_with_revenue,revenue
0,2019-05-01,2019-05-01,1672,2357.580
1,2019-05-02,2019-05-01,1420,514.679
2,2019-05-02,2019-05-02,1367,1629.140
3,2019-05-03,2019-05-01,1048,390.952
4,2019-05-03,2019-05-02,1164,533.608


In [18]:
# Переименуем стобик date в ctivity_date
revenue = revenue.rename(columns={'date':'activity_date'})

# Объедним таблицу retention_tabl и revenue по registration_date и activity_date
retention_table_with_revenue = retention_table.merge(revenue,on=['registration_date','activity_date'],how='left')
retention_table_with_revenue.head()

,activity_date,registration_date,active_users_count,registered_users_count,lifetime,users_count_with_revenue,revenue
0,2019-05-01,2019-05-01,2590,6516,0,1672.0,2357.580
1,2019-05-02,2019-05-01,2146,6516,1,1420.0,514.679
2,2019-05-02,2019-05-02,2049,5309,0,1367.0,1629.140
3,2019-05-03,2019-05-01,1637,6516,2,1048.0,390.952
4,2019-05-03,2019-05-02,1817,5309,1,1164.0,533.608


In [19]:
# Заполним пустые значения в стобиках revenue и sers_count_with_revenue значениями 0

for col in ['revenue','users_count_with_revenue']:
    retention_table_with_revenue[col] = retention_table_with_revenue[col].fillna(0)
    
# Изменим тип данных в users_count_with_revenue на целочисленные
retention_table_with_revenue['users_count_with_revenue'] = retention_table_with_revenue['users_count_with_revenue'].astype(int)
retention_table_with_revenue.head()

,activity_date,registration_date,active_users_count,registered_users_count,lifetime,users_count_with_revenue,revenue
0,2019-05-01,2019-05-01,2590,6516,0,1672,2357.580
1,2019-05-02,2019-05-01,2146,6516,1,1420,514.679
2,2019-05-02,2019-05-02,2049,5309,0,1367,1629.140
3,2019-05-03,2019-05-01,1637,6516,2,1048,390.952
4,2019-05-03,2019-05-02,1817,5309,1,1164,533.608


# Расчеты метрик 

## Retention

In [20]:
# Фильтруем таблицу по retention 7 дня для пользователей, привлеченных до релиза (май 2019)
retention_7_may_table = retention_table_with_revenue[retention_table_with_revenue['lifetime']==7][retention_table_with_revenue['registration_date']>="2019-05-01"][retention_table_with_revenue['registration_date']<"2019-06-01"]
retention_7_may_table.head()

C:\Users\sgari\AppData\Local\Temp\ipykernel_22396\736673720.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  retention_7_may_table = retention_table_with_revenue[retention_table_with_revenue['lifetime']==7][retention_table_with_revenue['registration_date']>="2019-05-01"][retention_table_with_revenue['registration_date']<"2019-06-01"]


,activity_date,registration_date,active_users_count,registered_users_count,lifetime,users_count_with_revenue,revenue
28,2019-05-08,2019-05-01,953,6516,7,616,63.9066
37,2019-05-09,2019-05-02,745,5309,7,490,313.7230
47,2019-05-10,2019-05-03,895,6266,7,571,110.7260
58,2019-05-11,2019-05-04,1186,8639,7,750,124.8590
70,2019-05-12,2019-05-05,1358,9787,7,885,174.9240


In [21]:
# Выполняем расчет retention 7 дня для этих когорт
retention_7_may = retention_7_may_table['active_users_count'].sum() / retention_7_may_table['registered_users_count'].sum()
print(retention_7_may)

0.12933000957960866


Retention 7  дня в мае 2019 г для пользователей, привлеченных до релиза = 12,9%

In [26]:
# Произведем аналогичные расчеты retention 7 дня пля пользователей привлеченных после релиза фичи в июне 2019.

retention_7_june_table = retention_table_with_revenue[retention_table_with_revenue['lifetime']==7][retention_table_with_revenue['registration_date']>="2019-06-01"][retention_table_with_revenue['registration_date']<"2019-07-01"]
retention_7_june_table.head()

C:\Users\sgari\AppData\Local\Temp\ipykernel_22396\1964290721.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  retention_7_june_table = retention_table_with_revenue[retention_table_with_revenue['lifetime']==7][retention_table_with_revenue['registration_date']>="2019-06-01"][retention_table_with_revenue['registration_date']<"2019-07-01"]


,activity_date,registration_date,active_users_count,registered_users_count,lifetime,users_count_with_revenue,revenue
736,2019-06-08,2019-06-01,545,4833,7,365,176.3160
767,2019-06-09,2019-06-02,683,5255,7,461,193.8390
798,2019-06-10,2019-06-03,484,4193,7,320,27.7548
829,2019-06-11,2019-06-04,511,4194,7,320,54.9610
860,2019-06-12,2019-06-05,472,3998,7,301,63.1554


In [27]:
retention_7_june = retention_7_june_table['active_users_count'].sum() / retention_7_june_table['registered_users_count'].sum()
print(retention_7_june)

0.12560867530346634


Retention 7 дня в июне 2019 г для пользователей, привлеченных после релиза = 12,6% - незначительно ниже показателя до введения фичи. Очевидно что новая фича никак не повлияла на показатель retention.

##  ARPPU

Теперь расчитаем ARPPU до и после (считаем средний доход именно на платящих клиентов, поэтому делить будем на 'users_count_with_revenue')

In [28]:
may_arppu = retention_table_with_revenue['revenue'][retention_table_with_revenue['registration_date']>="2019-05-01"][retention_table_with_revenue['registration_date']<"2019-06-01"].sum() / retention_table_with_revenue['users_count_with_revenue'][retention_table_with_revenue['registration_date']>="2019-05-01"][retention_table_with_revenue['registration_date']<"2019-06-01"].sum()
print(may_arppu)

0.3061245846810113


До релиза в мае 2019 года доход на одного платящего пользователя ARPPU составил 0,3 у.е.

In [33]:
june_arppu = retention_table_with_revenue['revenue'][retention_table_with_revenue['registration_date']>="2019-06-01"][retention_table_with_revenue['registration_date']<"2019-07-01"].sum() / retention_table_with_revenue['users_count_with_revenue'][retention_table_with_revenue['registration_date']>="2019-06-01"][retention_table_with_revenue['registration_date']<"2019-07-01"].sum()
print(june_arppu)
print((june_arppu/may_arppu-1)*100)

0.39319399168570485
28.442474522398076


После релиза фичи в июне 2019 года доход на одного платащего пользователя ARPPU увеличисля на 28,4% и составил 0,4 у.е.

# ВЫВОДЫ

1. Новая фича не повлияла на возвращаемость/лояльность покупателей.
2. После ввода новой фичи увеличился доход на 1 платащего пользователя на 28,4% с уровня 0.3 до 0.4 у.е.